In [1]:
targetFolderHBK = r"G:\Semester Arbeit\Programming\Extracted_Features\HBK\HBK_14285Hz_original_all_features\features"
targetFolderMCC5 = r"G:\Semester Arbeit\Programming\Extracted_Features\MCC5\MCC5_12800Hz_original_all_features_motor_vibration_x\features"
targetFolderSIZA = r"G:\Semester Arbeit\Programming\Extracted_Features\SIZA\SIZA_original_all_features\features"
normalization_method = "robust_scaling"

# Environment Setup & Imports

In [2]:
import os
import pandas as pd
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle
import numpy as np
import mlflow
from pytorch_lightning.loggers import MLFlowLogger
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score, balanced_accuracy_score, hamming_loss, hinge_loss, jaccard_score, log_loss, precision_score, recall_score, f1_score, make_scorer
from pathlib import Path
from pycaret.classification import * 
from torch import tensor
from torchmetrics.classification import BinaryAccuracy, MulticlassAccuracy
import optuna
import torch
from sklearn.model_selection import train_test_split
from pytorch_tabular import TabularModel
from pytorch_tabular.models import GANDALFConfig, CategoryEmbeddingModel,GatedAdditiveTreeEnsembleConfig, NodeConfig, FTTransformerConfig, TabNetModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    ModelConfig,
    TrainerConfig,
    ExperimentConfig,
)
from collections import Counter
from data_loader import load_feature_data

## Helper Functions

In [3]:
def normalizeDataframe(dataframe, normalization_method):
    """
    Normalizes the features of a dataframe using a specified method.

    Args:
        dataframe (pd.DataFrame): The input dataframe with a 'Label' column.
        normalization_method (str): The method to use ("min_max", "z_score", "robust_scaling").

    Returns:
        pd.DataFrame: The dataframe with scaled features.
    """
    # Separate features (X) and the target variable (y)
    y = dataframe['Label']
    X = dataframe.drop(columns=['Label'])

    # Select the scaler based on the chosen method
    if normalization_method == "min_max":
        scaler = MinMaxScaler()
    elif normalization_method == "z_score":
        scaler = StandardScaler()
    elif normalization_method == "robust_scaling":
        scaler = RobustScaler()
    else:
        # Raise an error for an invalid method name
        raise ValueError(f"Unknown normalization_method: '{normalization_method}'")

    # Fit the scaler to the data and transform it
    X_scaled = pd.DataFrame(
        scaler.fit_transform(X),
        columns=X.columns,
        index=X.index
    )

    # Rejoin the scaled features with the label column
    df_scaled = X_scaled.join(y)
    
    return df_scaled

In [4]:
def plotPredictionHistograms(df, domain, normalization):
    # 1) mark correct vs incorrect
    df = df.copy()
    df['prediction_quality'] = np.where(
        df['Label'] == df['prediction_label'],
        'correct',
        'incorrect'
    )
    
    # 2) choose a palette (you can override these colors if you like)
    pal = dict(zip(
        ['correct','incorrect'],
        sns.color_palette(n_colors=2)
    ))
    
    skip = {'Label','prediction_label','prediction_score','prediction_quality'}
    for col in df.columns:
        if col in skip:
            continue
        
        fig, ax = plt.subplots(figsize=(8,4))
        sns.histplot(
            data=df, x=col, hue='prediction_quality',
            palette=pal,
            kde=True, multiple='layer', element='step',
            alpha=0.5,
            ax=ax
        )
        
        # 3) build a manual legend using the same palette
        handles = [
            mpatches.Patch(color=pal[k], label=k)
            for k in ['correct','incorrect']
        ]
        ax.legend(
            handles=handles,
            title='Prediction Quality'
        )
        
        ax.set_title(
            f"Distribution of {col} in the '{domain}' domain\n"
            f"(normalization = '{normalization}')"
        )
        ax.set_xlabel(col)
        ax.set_ylabel("Count")
        plt.tight_layout()
        plt.show()

In [5]:
def get_incorrect_predictions(df):
    return df[
        ((df['Label'] == 'damaged')   & (df['prediction_label'] == 'healthy'))
      | ((df['Label'] == 'healthy')  & (df['prediction_label'] == 'damaged'))
    ].copy()

In [6]:
def get_feature_importance_df(model, df):
    importance = model.feature_importances_
    n = len(importance)
    features = df.columns[:n]
    fi_df = pd.DataFrame({
        'Features': features,
        'importance': importance
    })
    return fi_df.sort_values(by='importance', ascending=False).reset_index(drop=True)

In [7]:
def get_svm_feature_importance_df(model, df):
    if not hasattr(model, 'coef_'):
        raise ValueError("This SVM model has no coefficients. Use a linear kernel.")
    
    importance = model.coef_.ravel()  # Flatten in case of binary classification
    n = len(importance)
    features = df.columns[:n]
    fi_df = pd.DataFrame({
        'Features': features,
        'importance': abs(importance)
    })
    return fi_df.sort_values(by='importance', ascending=False).reset_index(drop=True)


In [8]:
feature_counter = Counter()
def add_top_features(feature_df: pd.DataFrame, top_n: int):
    top_features = feature_df.nlargest(top_n, 'importance')['Features']
    feature_counter.update(top_features)
    
def plot_feature_importance():
    feature_freq = pd.DataFrame(feature_counter.items(), columns=['Feature', 'Count'])
    plt.figure(figsize=(10, 5))
    sns.barplot(data=feature_freq.sort_values(by='Count', ascending=False),
                x='Feature', y='Count')
    plt.xticks(rotation=45)
    plt.title('Feature Frequency Across Experiments')
    plt.tight_layout()
    plt.show()

## Import Dataset

In [9]:
experiment_name = "Best_Hyperparameters_z_score"

In [10]:
df_binary_HBK = load_feature_data(
    features_path=targetFolderHBK,
    include_augmentations=False,      # Only 'original' data
    include_speed_torque=False,       # Drop operating conditions
    binary_classification=True,       # 'healthy' vs 'damaged'
)

Successfully loaded 161 files into a DataFrame with shape (200093, 30)
Applied binary classification: 'healthy' vs 'damaged'.
Dropped 'Speed' and 'Torque' columns.
Final DataFrame shape: (200093, 28)


In [11]:
df_binary_SIZA = load_feature_data(
    features_path=targetFolderSIZA,
    include_augmentations=False,      # Only 'original' data
    include_speed_torque=False,       # Drop operating conditions
    binary_classification=True,       # 'healthy' vs 'damaged'
)

Successfully loaded 36 files into a DataFrame with shape (53928, 30)
Applied binary classification: 'healthy' vs 'damaged'.
Dropped 'Speed' and 'Torque' columns.
Final DataFrame shape: (53928, 28)


In [12]:
df_binary_MCC5 = load_feature_data(
    features_path=targetFolderMCC5,
    include_augmentations=False,      # Only 'original' data
    include_speed_torque=False,       # Drop operating conditions
    binary_classification=True,       # 'healthy' vs 'damaged'
)

Successfully loaded 36 files into a DataFrame with shape (53928, 30)
Applied binary classification: 'healthy' vs 'damaged'.
Dropped 'Speed' and 'Torque' columns.
Final DataFrame shape: (53928, 28)


In [13]:
combined_df = pd.concat([df_binary_HBK, df_binary_SIZA, df_binary_MCC5], ignore_index=True)

In [14]:
normalized_df = normalizeDataframe(combined_df, normalization_method)

In [15]:
features_df_training_normalized, features_df_testing_normalized = train_test_split(
    normalized_df, 
    test_size=0.2,    # e.g., 20% for testing
    random_state=42   # for reproducibility
)

# Experiment Setup (ML)

## Setup Hyperparameters

In [16]:
experiment = setup(features_df_training_normalized, target='Label', log_experiment = True, experiment_name = experiment_name, use_gpu = True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3070, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that mee

,Description,Value
0,Session id,7641
1,Target,Label
2,Target type,Binary
3,Target mapping,"damaged: 0, healthy: 1"
4,Original data shape,"(246359, 28)"
5,Transformed data shape,"(246359, 28)"
6,Transformed train set shape,"(172451, 28)"
7,Transformed test set shape,"(73908, 28)"
8,Numeric features,27
9,Preprocess,True


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3070, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that mee

## Add aditional metrics

In [17]:
# Binary classification metrics
add_metric('balanced_acc', 'Balance Acc', balanced_accuracy_score, target='pred', greater_is_better=True)
add_metric('hamming_loss', 'Hamming Loss', hamming_loss, target='pred', greater_is_better=False)
add_metric('jaccard_score', 'Jaccard Score', jaccard_score, target='pred', greater_is_better=True)
add_metric('log_loss', 'Log Loss', log_loss, target='pred_proba', greater_is_better=False)

Name                                                          Log Loss
Display Name                                                  Log Loss
Score Function       <pycaret.internal.metrics.EncodedDecodedLabels...
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: log_loss, dtype: object

In [18]:
all_metrics = get_metrics()
all_metrics

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x0000022812267370>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(roc_auc_score, response_method=('d...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
recall,Recall,Recall,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(recall_score, response_method='pre...",pred,{'average': 'weighted'},True,True,False
precision,Precision,Prec.,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(precision_score, response_method='...",pred,{'average': 'weighted'},True,True,False
f1,F1,F1,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(f1_score, response_method='predict...",pred,{'average': 'weighted'},True,True,False
kappa,Kappa,Kappa,<function cohen_kappa_score at 0x0000022812267...,"make_scorer(cohen_kappa_score, response_method...",pred,{},True,True,False
mcc,MCC,MCC,<function matthews_corrcoef at 0x0000022812267...,"make_scorer(matthews_corrcoef, response_method...",pred,{},True,True,False
balanced_acc,Balance Acc,Balance Acc,<pycaret.internal.metrics.EncodedDecodedLabels...,"make_scorer(balanced_accuracy_score, response_...",pred,{},True,True,True
hamming_loss,Hamming Loss,Hamming Loss,<pycaret.internal.metrics.EncodedDecodedLabels...,"make_scorer(hamming_loss, greater_is_better=Fa...",pred,{},False,True,True


In [19]:
num_iterations_tuning = 10
optimized_metric = 'F1'

## Light Gradient Boosting Machine

In [20]:
lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9533,0.9868,0.9533,0.9541,0.9518,0.8579,0.8632,0.9044,0.0467,0.7971,0.1129
1,0.9509,0.9861,0.9509,0.9518,0.9493,0.8505,0.8562,0.8999,0.0491,0.7874,0.1162
2,0.9528,0.9870,0.9528,0.9538,0.9512,0.8561,0.8619,0.9025,0.0472,0.7945,0.1139
3,0.9532,0.9870,0.9532,0.9542,0.9517,0.8575,0.8631,0.9037,0.0468,0.7965,0.1132
4,0.9519,0.9870,0.9519,0.9531,0.9501,0.8528,0.8592,0.8998,0.0481,0.7901,0.1149
5,0.9540,0.9871,0.9540,0.9549,0.9525,0.8603,0.8654,0.9059,0.0460,0.8003,0.1118
6,0.9528,0.9866,0.9528,0.9538,0.9512,0.8562,0.8619,0.9027,0.0472,0.7947,0.1155
7,0.9555,0.9876,0.9555,0.9561,0.9542,0.8653,0.8697,0.9101,0.0445,0.8071,0.1092
8,0.9519,0.9869,0.9519,0.9527,0.9503,0.8535,0.8590,0.9018,0.0481,0.7914,0.1136


2025/09/26 16:37:35 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [21]:
lightgbm_tuned_model, lightgbm_tuner = tune_model(lightgbm, search_library = 'optuna', return_tuner=True, n_iter=num_iterations_tuning, optimize=optimized_metric)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9636,0.9921,0.9636,0.9637,0.9628,0.8916,0.8938,0.9291,0.0364,0.8427,0.0899
1,0.9604,0.9912,0.9604,0.9607,0.9595,0.8814,0.8843,0.9219,0.0396,0.8288,0.0945
2,0.9607,0.9917,0.9607,0.9608,0.9598,0.8827,0.8852,0.9238,0.0393,0.8307,0.0928
3,0.9615,0.9922,0.9615,0.9617,0.9607,0.8850,0.8876,0.9247,0.0385,0.8338,0.0908
4,0.9620,0.9921,0.9620,0.9623,0.9611,0.8864,0.8892,0.9246,0.0380,0.8354,0.0922
5,0.9629,0.9918,0.9629,0.9631,0.9621,0.8892,0.8918,0.9268,0.0371,0.8393,0.0909
6,0.9624,0.9918,0.9624,0.9625,0.9616,0.8878,0.8902,0.9267,0.0376,0.8375,0.0923
7,0.9648,0.9924,0.9648,0.9649,0.9641,0.8954,0.8974,0.9320,0.0352,0.8480,0.0878
8,0.9614,0.9917,0.9614,0.9616,0.9606,0.8848,0.8874,0.9245,0.0386,0.8335,0.0924


[LightGBM] [Warning] feature_fraction is set=0.5437803609036268, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5437803609036268
[LightGBM] [Warning] bagging_fraction is set=0.4228228034056274, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4228228034056274
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.5437803609036268, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5437803609036268
[LightGBM] [Warning] bagging_fraction is set=0.4228228034056274, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4228228034056274
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Number of positive: 34940, number of negative: 120265
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 6885
[LightGBM] [Info] Number of data

2025/09/26 16:55:58 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [22]:
print(lightgbm_tuned_model)

LGBMClassifier(bagging_fraction=0.9708703362894384, bagging_freq=2,
               boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', feature_fraction=0.507007220773583,
               importance_type='split', learning_rate=0.09688952429372566,
               max_depth=-1, min_child_samples=33, min_child_weight=0.001,
               min_split_gain=0.484949793073682, n_estimators=299, n_jobs=-1,
               num_leaves=49, objective=None, random_state=7641,
               reg_alpha=3.5114421971391514e-05,
               reg_lambda=0.000615806794641032, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


In [55]:
predictions_lightgbm = predict_model(lightgbm_tuned_model, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,Light Gradient Boosting Machine,0.9617,0.9920,0.9617,0.9620,0.9609,0.8860,0.8887,0.9250,0.0383,0.8353,0.0914


[LightGBM] [Warning] feature_fraction is set=0.507007220773583, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.507007220773583
[LightGBM] [Warning] bagging_fraction is set=0.9708703362894384, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9708703362894384
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.507007220773583, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.507007220773583
[LightGBM] [Warning] bagging_fraction is set=0.9708703362894384, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9708703362894384
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


In [23]:
evaluate_model(lightgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [24]:
lightgbm_top_features = get_feature_importance_df(lightgbm, features_df_training_normalized)
lightgbm_top_features

,Features,importance
0,meanFreq,303
1,bandwidth,300
2,medianFreq,289
3,meanWavelet,278
4,spectral_flatness,271
5,skewness,191
6,spectral_skewness,154
7,entropyWavelet,150
8,spectral_kurtosis,130
9,varWavelet,122


In [25]:
predictions_lightgbm = predict_model(lightgbm, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,Light Gradient Boosting Machine,0.9518,0.9866,0.9518,0.9526,0.9502,0.8539,0.8592,0.9024,0.0482,0.7921,0.1144


In [26]:
predictions_lightgbm

,mean,rms,std,skewness,kurtosis,ptp,crest,impulse,clearance,shape,...,varWavelet,entropyWavelet,energyWavelet,meanSpectrogram,varSpectrogram,entropySpectrogram,energySpectrogram,Label,prediction_label,prediction_score
93960,6.357815,2.343891,2.344943,0.635143,0.131432,2.410167,0.497478,0.455357,1.546793,0.243375,...,3.512821,-0.984801,5.694981,5.858073,3.476256,5.900208,5.697627,damaged,damaged,0.9999
371,-1.272307,-0.564226,-0.568537,1.528791,0.216020,-0.510851,0.125702,0.104172,-0.696706,0.053627,...,-0.280870,-0.318820,-0.302703,-0.292659,-0.080855,-0.093596,-0.079652,healthy,damaged,0.9059
33266,0.645020,-0.632693,-0.633274,-0.551617,-0.094884,-0.575886,-0.487928,-0.467216,-0.944098,-0.281379,...,-0.282883,-0.354399,-0.306426,-0.297714,-0.080904,-0.093679,-0.079699,healthy,healthy,0.8216
238774,-0.578217,0.212273,0.214511,0.427125,0.010503,0.290723,0.816735,0.688420,0.395715,0.021394,...,0.271075,0.898755,0.259001,0.245505,0.127312,0.152609,0.132502,damaged,damaged,0.9465
210317,-0.568005,-0.318859,-0.317333,0.193032,0.447690,-0.287186,-0.137432,0.016710,-0.330195,0.947099,...,-0.194162,-0.020849,-0.221469,-0.218766,-0.069007,-0.078770,-0.068269,damaged,damaged,0.9242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241132,-0.461992,-0.303484,-0.301819,-0.136055,2.333936,-0.163282,1.232286,1.508070,-0.000384,2.301808,...,-0.180947,-0.034000,-0.213441,-0.202451,-0.056868,-0.065837,-0.056839,damaged,damaged,0.8141
214791,-0.497348,-0.266087,-0.264392,-0.137965,2.325956,-0.112111,1.130247,1.260849,0.006792,1.544519,...,-0.164856,0.034796,-0.192630,-0.176194,-0.042728,-0.051209,-0.043369,damaged,damaged,0.8668
165165,2.155673,1.023697,1.026224,1.294237,1.292390,1.168869,0.929289,1.075880,1.148403,1.532599,...,2.056622,-1.219416,1.771152,1.718675,11.675074,8.020503,11.074836,damaged,damaged,0.9998
217690,-0.375742,0.747205,0.750013,1.155028,-0.124393,0.656685,-0.064616,-0.092430,0.465135,-0.163915,...,1.155234,0.974433,1.168414,1.209354,2.298496,2.102510,2.272903,damaged,damaged,0.6165


In [27]:
get_incorrect_predictions(predictions_lightgbm)

,mean,rms,std,skewness,kurtosis,ptp,crest,impulse,clearance,shape,...,varWavelet,entropyWavelet,energyWavelet,meanSpectrogram,varSpectrogram,entropySpectrogram,energySpectrogram,Label,prediction_label,prediction_score
371,-1.272307,-0.564226,-0.568537,1.528791,0.216020,-0.510851,0.125702,0.104172,-0.696706,0.053627,...,-0.280870,-0.318820,-0.302703,-0.292659,-0.080855,-0.093596,-0.079652,healthy,damaged,0.9059
258524,-0.462637,-0.614765,-0.615758,-1.082147,1.396836,-0.548946,1.235335,1.315180,-0.751774,1.342675,...,-0.282070,-0.345832,-0.305536,-0.296682,-0.080900,-0.093672,-0.079696,healthy,damaged,0.6379
207745,-0.473546,0.357993,0.360417,-0.554096,-0.787129,0.162519,-0.831262,-0.831097,-0.045980,-0.864700,...,0.543103,0.665827,0.465029,0.464907,0.870633,0.803324,0.839726,healthy,damaged,0.7560
255723,-0.490795,0.123172,0.125355,0.375499,-0.076538,0.076991,-0.343168,-0.295428,-0.008315,0.060085,...,0.171299,0.706681,0.148892,0.159414,0.092594,0.108320,0.094701,healthy,damaged,0.7437
207040,-0.353068,0.308735,0.311149,0.414371,0.260009,0.414386,0.282649,0.278528,0.326729,0.291802,...,0.395646,1.035493,0.392030,0.422492,0.317823,0.349905,0.323606,healthy,damaged,0.5595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206916,-0.283023,0.173179,0.175487,0.596202,-0.180180,0.145434,-0.281675,-0.259991,0.039367,-0.055206,...,0.222261,0.832072,0.209463,0.246596,0.172595,0.192210,0.174519,healthy,damaged,0.9315
200497,-0.576046,-0.231117,-0.229435,0.114024,0.946886,-0.155104,0.381899,0.562282,-0.103635,1.438474,...,-0.131645,0.055914,-0.171216,-0.158665,-0.042632,-0.049115,-0.042980,healthy,damaged,0.6273
258687,-0.421962,-0.277288,-0.275546,0.249954,0.933059,-0.222218,-0.017272,0.138013,-0.257450,1.040160,...,-0.162309,0.002188,-0.199004,-0.182658,-0.051269,-0.058800,-0.051380,healthy,damaged,0.6073
205706,-0.496038,-0.302834,-0.301201,0.114997,0.769237,-0.220468,0.313372,0.362557,-0.235371,0.629632,...,-0.184069,0.001882,-0.213110,-0.198862,-0.062567,-0.071403,-0.062071,healthy,damaged,0.6771


## Random Forest Classifier

In [28]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9780,0.9959,0.9780,0.9782,0.9777,0.9351,0.9363,0.9553,0.0220,0.9033,0.0746
1,0.9754,0.9950,0.9754,0.9755,0.9750,0.9273,0.9286,0.9507,0.0246,0.8922,0.0754
2,0.9760,0.9953,0.9760,0.9761,0.9757,0.9293,0.9305,0.9522,0.0240,0.8950,0.0742
3,0.9758,0.9958,0.9758,0.9760,0.9755,0.9288,0.9299,0.9520,0.0242,0.8943,0.0736
4,0.9763,0.9960,0.9763,0.9765,0.9759,0.9300,0.9313,0.9519,0.0237,0.8960,0.0732
5,0.9773,0.9953,0.9773,0.9775,0.9769,0.9331,0.9342,0.9541,0.0227,0.9003,0.0743
6,0.9749,0.9953,0.9749,0.9751,0.9746,0.9262,0.9274,0.9505,0.0251,0.8906,0.0740
7,0.9782,0.9957,0.9782,0.9783,0.9779,0.9360,0.9369,0.9567,0.0218,0.9045,0.0722
8,0.9754,0.9953,0.9754,0.9755,0.9750,0.9274,0.9286,0.9510,0.0246,0.8923,0.0769


2025/09/26 16:58:48 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [29]:
rf_tuned_model, rf_tuner = tune_model(rf, search_library = 'optuna', return_tuner=True, n_iter=num_iterations_tuning, optimize=optimized_metric)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9396,0.9766,0.9396,0.9387,0.9383,0.8198,0.8221,0.8940,0.0604,0.7514,0.1907
1,0.9400,0.9755,0.9400,0.9392,0.9385,0.8201,0.8229,0.8920,0.0600,0.7511,0.1937
2,0.9366,0.9755,0.9366,0.9355,0.9352,0.8108,0.8130,0.8896,0.0634,0.7405,0.1932
3,0.9361,0.9756,0.9361,0.9350,0.9348,0.8096,0.8117,0.8895,0.0639,0.7392,0.1941
4,0.9396,0.9771,0.9396,0.9390,0.9379,0.8178,0.8215,0.8887,0.0604,0.7479,0.1926
5,0.9352,0.9762,0.9352,0.9344,0.9347,0.8113,0.8116,0.8994,0.0648,0.7434,0.1903
6,0.9402,0.9756,0.9402,0.9394,0.9387,0.8206,0.8235,0.8923,0.0598,0.7518,0.1916
7,0.9395,0.9766,0.9395,0.9387,0.9382,0.8192,0.8217,0.8928,0.0605,0.7504,0.1909
8,0.9424,0.9773,0.9424,0.9420,0.9409,0.8265,0.8301,0.8931,0.0576,0.7586,0.1925


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


2025/09/26 18:09:47 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [30]:
print(rf_tuned_model)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=100, n_jobs=-1,
                       oob_score=False, random_state=7641, verbose=0,
                       warm_start=False)


In [56]:
predictions_rf = predict_model(rf_tuned_model, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,Random Forest Classifier,0.9787,0.9961,0.9787,0.9788,0.9784,0.9376,0.9385,0.9574,0.0213,0.9069,0.0714


In [31]:
evaluate_model(rf)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [32]:
rf_top_features = get_feature_importance_df(rf, features_df_training_normalized)
rf_top_features

,Features,importance
0,meanWavelet,0.117837
1,medianFreq,0.079955
2,energyWavelet,0.058169
3,meanFreq,0.054457
4,spectral_entropy,0.052832
5,std,0.050229
6,spectral_flatness,0.046179
7,bandwidth,0.046031
8,meanSpectrogram,0.045034
9,energy,0.040885


In [33]:
predictions_rf = predict_model(rf, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,Random Forest Classifier,0.9787,0.9961,0.9787,0.9788,0.9784,0.9376,0.9385,0.9574,0.0213,0.9069,0.0714


In [34]:
get_incorrect_predictions(predictions_rf)

,mean,rms,std,skewness,kurtosis,ptp,crest,impulse,clearance,shape,...,varWavelet,entropyWavelet,energyWavelet,meanSpectrogram,varSpectrogram,entropySpectrogram,energySpectrogram,Label,prediction_label,prediction_score
255723,-0.490795,0.123172,0.125355,0.375499,-0.076538,0.076991,-0.343168,-0.295428,-0.008315,0.060085,...,0.171299,0.706681,0.148892,0.159414,0.092594,0.108320,0.094701,healthy,damaged,0.63
207040,-0.353068,0.308735,0.311149,0.414371,0.260009,0.414386,0.282649,0.278528,0.326729,0.291802,...,0.395646,1.035493,0.392030,0.422492,0.317823,0.349905,0.323606,healthy,damaged,0.69
215923,-0.566900,0.279868,0.282180,-0.605398,-0.248029,0.227013,-0.263786,-0.248145,0.118507,-0.077784,...,0.367203,0.918918,0.350482,0.403667,0.371225,0.392265,0.371312,damaged,healthy,0.55
202856,-0.671359,0.200738,0.202919,0.259419,0.332430,0.226696,0.075642,0.120797,0.182180,0.421321,...,0.258286,0.854640,0.244118,0.267044,0.189892,0.209489,0.192026,healthy,damaged,0.83
257105,-0.491878,0.020361,0.022433,0.302523,0.527255,0.101222,0.478098,0.552123,0.168857,0.886951,...,0.107578,0.279084,0.036760,0.062195,0.125821,0.128799,0.120267,healthy,damaged,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202304,-0.467441,0.098287,0.100455,0.445940,0.340490,0.125023,-0.030906,0.021997,0.072717,0.381093,...,0.145469,0.667298,0.120250,0.161309,0.120162,0.130844,0.120347,healthy,damaged,0.78
200447,-0.460936,-0.285567,-0.283871,0.693839,2.795959,-0.113762,1.463045,1.744825,0.078182,2.398688,...,-0.169029,-0.022774,-0.203725,-0.189921,-0.048459,-0.057060,-0.048883,healthy,damaged,0.74
259550,-0.322300,-0.268401,-0.266572,0.080743,0.542818,-0.170980,0.595511,0.592653,-0.137069,0.536099,...,-0.161250,0.043342,-0.193911,-0.179916,-0.054774,-0.062395,-0.054584,healthy,damaged,0.80
210944,-0.591265,0.185384,0.187587,-0.365052,-0.196839,0.123479,-0.525343,-0.477898,-0.026761,-0.132084,...,0.240576,0.872083,0.224674,0.261468,0.146130,0.172737,0.151084,damaged,healthy,0.52


## SVM

In [35]:
svm = create_model('svm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.8923,0.9103,0.8923,0.8905,0.8853,0.6557,0.6707,0.7939,0.1077,0.5624,-0.0000
1,0.8980,0.9102,0.8980,0.8959,0.8923,0.6783,0.6899,0.8082,0.1020,0.5873,-0.0000
2,0.8999,0.9078,0.8999,0.8973,0.8952,0.6883,0.6970,0.8168,0.1001,0.5995,-0.0000
3,0.7855,0.7915,0.7855,0.7985,0.7909,0.4183,0.4203,0.7210,0.2145,0.3879,-0.0000
4,0.8429,0.8320,0.8429,0.8379,0.8399,0.5331,0.5343,0.7571,0.1571,0.4627,-0.0000
5,0.9017,0.9075,0.9017,0.8993,0.8971,0.6941,0.7027,0.8196,0.0983,0.6055,-0.0000
6,0.8972,0.9096,0.8972,0.8947,0.8920,0.6780,0.6880,0.8101,0.1028,0.5880,-0.0000
7,0.8903,0.9029,0.8903,0.8869,0.8877,0.6710,0.6735,0.8206,0.1097,0.5875,-0.0000
8,0.9035,0.9065,0.9035,0.9010,0.8992,0.7012,0.7087,0.8246,0.0965,0.6137,-0.0000


2025/09/26 18:10:07 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [36]:
svm_tuned_model, svm_tuner = tune_model(svm, search_library = 'optuna', return_tuner=True, n_iter=num_iterations_tuning, optimize=optimized_metric)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.8875,0.9062,0.8875,0.8840,0.8849,0.6636,0.6656,0.8181,0.1125,0.5806,-0.0000
1,0.8829,0.9004,0.8829,0.8791,0.8801,0.6492,0.6513,0.8106,0.1171,0.5662,-0.0000
2,0.8111,0.8123,0.8111,0.8120,0.8116,0.4611,0.4612,0.7316,0.1889,0.4117,-0.0000
3,0.8133,0.8151,0.8133,0.8145,0.8139,0.4683,0.4683,0.7357,0.1867,0.4175,-0.0000
4,0.8931,0.9182,0.8931,0.8895,0.8894,0.6739,0.6786,0.8165,0.1069,0.5879,-0.0000
5,0.8933,0.9152,0.8933,0.8898,0.8902,0.6775,0.6808,0.8212,0.1067,0.5928,-0.0000
6,0.8841,0.9031,0.8841,0.8806,0.8816,0.6541,0.6559,0.8139,0.1159,0.5714,-0.0000
7,0.8830,0.9107,0.8830,0.8800,0.8810,0.6534,0.6547,0.8161,0.1170,0.5719,-0.0000
8,0.9005,0.9187,0.9005,0.8975,0.8975,0.6985,0.7023,0.8303,0.0995,0.6139,-0.0000


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


2025/09/26 18:12:19 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [37]:
print(svm_tuned_model)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.001, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=7641, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)


In [57]:
predictions_svm = predict_model(svm_tuned_model, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,SVM - Linear Kernel,0.8889,0,0.8889,0.8863,0.8821,0.6478,0.6613,0.7918,0.1111,0.5558,0


In [38]:
evaluate_model(svm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [39]:
svm_top_features = get_svm_feature_importance_df(svm, features_df_training_normalized)
svm_top_features

,Features,importance
0,entropySpectrogram,16.222591
1,meanWavelet,14.193700
2,entropyWavelet,8.031571
3,rms,7.827267
4,std,6.623096
5,energySpectrogram,6.519983
6,varWavelet,5.241671
7,varSpectrogram,4.964353
8,ptp,4.807109
9,spectral_entropy,3.328294


In [40]:
predictions_svm = predict_model(svm, data=features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,SVM - Linear Kernel,0.8889,0,0.8889,0.8863,0.8821,0.6478,0.6613,0.7918,0.1111,0.5558,0


In [41]:
get_incorrect_predictions(predictions_svm)

,mean,rms,std,skewness,kurtosis,ptp,crest,impulse,clearance,shape,...,meanWavelet,varWavelet,entropyWavelet,energyWavelet,meanSpectrogram,varSpectrogram,entropySpectrogram,energySpectrogram,Label,prediction_label
371,-1.272307,-0.564226,-0.568537,1.528791,0.216020,-0.510851,0.125702,0.104172,-0.696706,0.053627,...,-0.532978,-0.280870,-0.318820,-0.302703,-0.292659,-0.080855,-0.093596,-0.079652,healthy,damaged
261113,-0.464289,0.389850,0.392307,0.236188,-0.322119,0.347214,-0.110471,-0.137664,0.232741,-0.205468,...,0.448827,0.510743,1.116157,0.514528,0.549057,0.468693,0.505024,0.476336,healthy,damaged
207745,-0.473546,0.357993,0.360417,-0.554096,-0.787129,0.162519,-0.831262,-0.831097,-0.045980,-0.864700,...,0.282735,0.543103,0.665827,0.465029,0.464907,0.870633,0.803324,0.839726,healthy,damaged
255723,-0.490795,0.123172,0.125355,0.375499,-0.076538,0.076991,-0.343168,-0.295428,-0.008315,0.060085,...,0.142535,0.171299,0.706681,0.148892,0.159414,0.092594,0.108320,0.094701,healthy,damaged
207040,-0.353068,0.308735,0.311149,0.414371,0.260009,0.414386,0.282649,0.278528,0.326729,0.291802,...,0.359017,0.395646,1.035493,0.392030,0.422492,0.317823,0.349905,0.323606,healthy,damaged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258687,-0.421962,-0.277288,-0.275546,0.249954,0.933059,-0.222218,-0.017272,0.138013,-0.257450,1.040160,...,-0.321609,-0.162309,0.002188,-0.199004,-0.182658,-0.051269,-0.058800,-0.051380,healthy,damaged
29271,0.422849,-0.305670,-0.303739,-0.067514,-1.538958,-0.378466,-1.560269,-1.551168,-0.647673,-2.065707,...,-0.351838,-0.178829,0.007978,-0.214714,-0.214982,-0.068415,-0.077448,-0.067680,healthy,damaged
205706,-0.496038,-0.302834,-0.301201,0.114997,0.769237,-0.220468,0.313372,0.362557,-0.235371,0.629632,...,-0.311886,-0.184069,0.001882,-0.213110,-0.198862,-0.062567,-0.071403,-0.062071,healthy,damaged
201271,-0.526296,-0.264085,-0.262413,-0.084871,0.369411,-0.234605,-0.231204,-0.123989,-0.298620,0.564308,...,-0.304420,-0.155067,0.028787,-0.191423,-0.181192,-0.052162,-0.059515,-0.052184,healthy,damaged


# Experiment Setup (DL)

## Configure Data

In [42]:
train, test = train_test_split(features_df_training_normalized, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)
print(f"Train Shape: {train.shape} | Val Shape: {val.shape} | Test Shape: {test.shape}")

Train Shape: (157669, 28) | Val Shape: (39418, 28) | Test Shape: (49272, 28)


In [43]:
target = "Label"

categorical_cols = [
    col
    for col in features_df_training_normalized.select_dtypes(include=["object","category"]).columns
    if col != target
]

continuous_cols = features_df_training_normalized.select_dtypes(include=["number"]).columns.tolist()

In [44]:
print("Target:", target)
print("Categorical inputs:", categorical_cols)  
print("Continuous inputs:", continuous_cols)    

Target: Label
Categorical inputs: []
Continuous inputs: ['mean', 'rms', 'std', 'skewness', 'kurtosis', 'ptp', 'crest', 'impulse', 'clearance', 'shape', 'energy', 'entropy', 'meanFreq', 'medianFreq', 'bandwidth', 'spectral_flatness', 'spectral_entropy', 'spectral_skewness', 'spectral_kurtosis', 'meanWavelet', 'varWavelet', 'entropyWavelet', 'energyWavelet', 'meanSpectrogram', 'varSpectrogram', 'entropySpectrogram', 'energySpectrogram']


In [45]:
data_config = DataConfig(
    target=[target],
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
)

In [46]:
available_gpu=1 if torch.cuda.is_available() else 0
print(f"Available GPU: {'Yes' if available_gpu else 'No'}")

Available GPU: Yes


In [47]:
trainer_config = TrainerConfig(
    auto_lr_find=True,
    max_epochs=20,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    batch_size=256,
)

optimizer_config = OptimizerConfig()

experiment_config = ExperimentConfig(
        project_name="TEST",
        run_name="test",
        log_target="tensorboard",
    )

In [48]:
n_trials = 10

## Tabnet

In [49]:
def TabNet_Optimization(trial):
    n_d     = trial.suggest_int("n_d", 4, 64)
    n_a     = trial.suggest_int("n_a", 4, 64)
    n_steps = trial.suggest_int("n_steps", 3, 10)
    gamma   = trial.suggest_float("gamma", 1.0, 2.0)
    embedding_dropout = trial.suggest_float("embedding_dropout", 0, 1)
    lr      = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)
    
    tabnet_config = TabNetModelConfig(
        task="classification",
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        embedding_dropout=embedding_dropout,
        learning_rate=lr,
        n_independent=2,
        metrics=[
            "auroc",
            "recall",
            "precision",
            "f1_score",
            "cohen_kappa",
            "matthews_corrcoef",
            "hamming_distance",
            "jaccard_index",
        ],
        metrics_prob_input=[
            True,   # auroc
            False,  # recall
            False,  # precision
            False,  # f1_score
            False,  # cohen_kappa
            False,  # matthews_corrcoef
            False,  # hamming_distance
            False,  # jaccard_index
        ],
        metrics_params=[
            {"average": "macro", "num_classes": 2},  # auroc
            {"average": "macro", "num_classes": 2},  # recall
            {"average": "macro", "num_classes": 2},  # precision
            {"average": "macro", "num_classes": 2},  # f1_score
            {"num_classes": 2},                      # cohen_kappa
            {},                                      # matthews_corrcoef
            {},                                      # hamming_distance
            {"average": "macro", "num_classes": 2},  # jaccard_index
        ]
    )
    
    tabnet_model = TabularModel(
        data_config=data_config,
        model_config=tabnet_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        verbose=True
    )
    
    tabnet_model.fit(train=train, validation=val)

    preds_df = tabnet_model.predict(val)
    y_pred = preds_df["Label_prediction"].to_numpy()
    y_true = val["Label"].to_numpy()
    return f1_score(y_true, y_pred, average="macro")

In [50]:
tabnet_study = optuna.create_study(direction="maximize")
tabnet_study.optimize(TabNet_Optimization, n_trials=n_trials)

2025-09-26 18:12:22,945 - {pytorch_tabular.tabular_model:146} - INFO - Experiment Tracking is turned off

Seed set to 42


2025-09-26 18:12:22,983 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-09-26 18:12:23,010 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
classification task

2025-09-26 18:12:23,171 - {pytorch_tabular.tabular_model:599} - INFO - Preparing the Model: TabNetModel

...

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _embedding_layer │ Identity         │      0 │ train │
│ 1 │ _backbone        │ TabNetBackbone   │  146 K │ train │
│ 2 │ _head            │ Identity         │      0 │ train │
│ 3 │ loss             │ CrossEntropyLoss │      0 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 146 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 146 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 203                                                                                         
Modules in eval mode: 0

Output()

2025-09-26 19:45:26,798 - {pytorch_tabular.tabular_model:689} - INFO - Training the model completed

2025-09-26 19:45:26,801 - {pytorch_tabular.tabular_model:1529} - INFO - Loading the best model

In [51]:
print("Best params:", tabnet_study.best_params)
print("Best F1 score:", tabnet_study.best_value)

Best params: {'n_d': 8, 'n_a': 25, 'n_steps': 6, 'gamma': 1.1285381196618185, 'embedding_dropout': 0.4518844821358271, 'learning_rate': 0.00010830099505845923}
Best F1 score: 0.8915709443898493


## GANDALF

In [52]:
def GANDALF_Optimization(trial):
    gflu_stages               = trial.suggest_int("gflu_stages", 1, 10)
    gflu_dropout              = trial.suggest_float("gflu_dropout", 0.0, 0.5)
    gflu_feature_init_sparsity = trial.suggest_float("gflu_feature_init_sparsity", 0.1, 0.9)
    learnable_sparsity        = trial.suggest_categorical("learnable_sparsity", [True, False])
    embedding_dropout         = trial.suggest_float("embedding_dropout", 0.0, 0.5)
    batch_norm_continuous     = trial.suggest_categorical("batch_norm_continuous_input", [True, False])
    learning_rate             = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)

    # 2) build Gandalf config
    gandalf_config = GANDALFConfig(
        task="classification",
        gflu_stages=gflu_stages,
        gflu_dropout=gflu_dropout,
        gflu_feature_init_sparsity=gflu_feature_init_sparsity,
        learnable_sparsity=learnable_sparsity,
        embedding_dropout=embedding_dropout,
        batch_norm_continuous_input=batch_norm_continuous,
        learning_rate=learning_rate,
        metrics=[
            "auroc",
            "recall",
            "precision",
            "f1_score",
            "cohen_kappa",
            "matthews_corrcoef",
            "hamming_distance",
            "jaccard_index",
        ],
        metrics_prob_input=[
            True,   # auroc
            False,  # recall
            False,  # precision
            False,  # f1_score
            False,  # cohen_kappa
            False,  # matthews_corrcoef
            False,  # hamming_distance
            False,  # jaccard_index
        ],
        metrics_params=[
            {"average": "macro", "num_classes": 2},  # auroc
            {"average": "macro", "num_classes": 2},  # recall
            {"average": "macro", "num_classes": 2},  # precision
            {"average": "macro", "num_classes": 2},  # f1_score
            {"num_classes": 2},                      # cohen_kappa
            {},                                      # matthews_corrcoef
            {},                                      # hamming_distance
            {"average": "macro", "num_classes": 2},  # jaccard_index
        ]
    )

    # 3) instantiate & train
    model = TabularModel(
        data_config=data_config,
        model_config=gandalf_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        verbose=True
    )
    model.fit(train=train, validation=val)

    # 4) predict & return macro-F1
    preds = model.predict(val)
    y_pred = preds["Label_prediction"].to_numpy()
    y_true = val["Label"].to_numpy()
    return f1_score(y_true, y_pred, average="macro")

In [53]:
gandalf_study = optuna.create_study(direction="maximize")
gandalf_study.optimize(GANDALF_Optimization, n_trials=n_trials)

2025-09-26 19:45:31,932 - {pytorch_tabular.tabular_model:146} - INFO - Experiment Tracking is turned off

Seed set to 42


2025-09-26 19:45:31,969 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-09-26 19:45:32,001 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
classification task

2025-09-26 19:45:32,161 - {pytorch_tabular.tabular_model:599} - INFO - Preparing the Model: GANDALFModel

...

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _backbone        │ GANDALFBackbone  │ 31.4 K │ train │
│ 1 │ _embedding_layer │ Embedding1dLayer │      0 │ train │
│ 2 │ _head            │ Sequential       │     58 │ train │
│ 3 │ loss             │ CrossEntropyLoss │      0 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 31.4 K                                                                                           
Non-trainable params: 7                                                                                            
Total params: 31.4 K                                                                                               
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 28                                                                                          
Modules in eval mode: 0

Output()

2025-09-26 20:31:11,382 - {pytorch_tabular.tabular_model:689} - INFO - Training the model completed

2025-09-26 20:31:11,384 - {pytorch_tabular.tabular_model:1529} - INFO - Loading the best model

In [54]:
print("Best params:", gandalf_study.best_params)
print("Best F1 score:", gandalf_study.best_value)

Best params: {'gflu_stages': 1, 'gflu_dropout': 0.39399066451650944, 'gflu_feature_init_sparsity': 0.21721669194200377, 'learnable_sparsity': True, 'embedding_dropout': 0.034740846371525747, 'batch_norm_continuous_input': False, 'learning_rate': 8.440408399291e-05}
Best F1 score: 0.9046138003493449
